In [ ]:
import pandas as pd
import numpy as np
import random
import os
import pickle
from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train = pickle.load(open('/content/drive/MyDrive/data_0728.pickle', 'rb'))

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

In [ ]:
#VWAP 계산 및 차이 컬럼 추가(차이 컬럼 기준으로 매수 매도 판단)
train['볼륨가격'] = train['거래량']*((train['시가']+train['고가']+train['저가'])/3)
train['볼륨가격합'] = train.groupby(['종목코드'])['볼륨가격'].apply(lambda x:x.cumsum())
train['볼륨합'] = train.groupby(['종목코드'])['거래량'].apply(lambda x:x.cumsum())
train['VWAP'] = train['볼륨가격합']/train['볼륨합']
train['차이'] = train['시가']-train['VWAP']

In [ ]:
train[train['볼륨가격'] == 0]

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률,종목코드,OBV,...,dow,snp500,nasdaq,semicon_index,euro50,볼륨가격,볼륨가격합,볼륨합,VWAP,VWAP_차이
4321,2023-06-29,0,0,0,1880000,0,0,0.0,048260,-63041,...,34122.42,4396.44,13591.33,3614.18,4354.69,0.0,1.426841e+11,75969,1.878188e+06,-1.878188e+06
4322,2023-06-30,0,0,0,1880000,0,0,0.0,048260,-63041,...,34407.60,4450.38,13787.92,3673.06,4399.09,0.0,1.426841e+11,75969,1.878188e+06,-1.878188e+06
4323,2023-07-03,0,0,0,1880000,0,0,0.0,048260,-63041,...,34418.47,4455.59,13816.77,3703.62,4398.15,0.0,1.426841e+11,75969,1.878188e+06,-1.878188e+06
4324,2023-07-04,0,0,0,1880000,0,0,0.0,048260,-63041,...,NaN,NaN,NaN,NaN,4390.99,0.0,1.426841e+11,75969,1.878188e+06,-1.878188e+06
4325,2023-07-05,0,0,0,1880000,0,0,0.0,048260,-63041,...,34288.64,4446.82,13791.65,3622.32,4350.71,0.0,1.426841e+11,75969,1.878188e+06,-1.878188e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85995,2023-07-24,0,0,0,0,0,0,0.0,096640,52770643,...,35411.24,4554.64,14058.87,3692.32,4383.03,0.0,1.422178e+10,64978982,2.188675e+02,-2.188675e+02
85996,2023-07-25,0,0,0,0,0,0,0.0,096640,52770643,...,35438.07,4567.46,14144.56,3755.27,4391.30,0.0,1.422178e+10,64978982,2.188675e+02,-2.188675e+02
85997,2023-07-26,0,0,0,0,0,0,0.0,096640,52770643,...,35520.12,4566.75,14127.28,3699.37,4346.15,0.0,1.422178e+10,64978982,2.188675e+02,-2.188675e+02
85998,2023-07-27,0,0,0,0,0,0,0.0,096640,52770643,...,35282.72,4537.41,14050.11,3768.27,4447.44,0.0,1.422178e+10,64978982,2.188675e+02,-2.188675e+02


In [ ]:
vwap_df = train[['날짜','종목코드','VWAP','VWAP_차이']]
vwap_df

,날짜,종목코드,VWAP,VWAP_차이
1,2023-05-31,005930,71777.086378,622.913622
2,2023-06-01,005930,71616.196145,-716.196145
3,2023-06-02,005930,71649.216577,50.783423
4,2023-06-05,005930,71733.755828,966.244172
5,2023-06-07,005930,71664.848990,-364.848990
...,...,...,...,...
85995,2023-07-24,096640,218.867464,-218.867464
85996,2023-07-25,096640,218.867464,-218.867464
85997,2023-07-26,096640,218.867464,-218.867464
85998,2023-07-27,096640,218.867464,-218.867464


In [ ]:
with open('vwap_df.pickle', 'wb') as f:
    pickle.dump(vwap_df, f)

In [ ]:
vwap_df.isna().sum()
vwap_df = vwap_df.fillna(0)
vwap_df.isna().sum()

날짜         0
종목코드       0
VWAP       0
VWAP_차이    0
dtype: int64

In [ ]:
# VWAP으로 구한 차이값 중 상위 200개의 종목 선정(매수)

# 차이값이 양수인 것(그냥 예시로 이렇게 잡아봤는데 200개 이상이어서 그냥 사용)
train_diff = train[train['VWAP_차이'] > 1]

#20230530일 선택
train_result = train_diff[train_diff['날짜'] == '2023-07-21']

buy_stock = train_result.sort_values('VWAP_차이')
buy_stock.reset_index(drop = True, inplace = True)
buy_stock = buy_stock[:200] #상위 200개
buy_stock = buy_stock[['종목코드']].reset_index()
buy_stock['index'] = buy_stock['index']+1 #rank로 사용하기 위해 +1
buy_stock.columns = ['순위','종목코드']
buy_stock

,순위,종목코드
0,1,102120
1,2,094970
2,3,002240
3,4,008040
4,5,037330
...,...,...
195,196,004170
196,197,242040
197,198,044820
198,199,098120


In [ ]:
# VWAP으로 구한 차이값 중 하위 200개의 종목 선정(매도)

#차이값이 음수인 것
train_diff_rev = train[train['VWAP_차이'] < -1] #reverse

#날짜 선택
train_result_rev = train_diff_rev[train_diff_rev['날짜'] == '2023-07-21']

sell_stock = train_result_rev.sort_values('VWAP_차이')
sell_stock.reset_index(drop = True, inplace = True)
sell_stock = sell_stock[:200] #하위 200개(음수기준 정렬)
sell_stock = sell_stock[['종목코드']].reset_index()
sell_stock['index'] = sell_stock['index']+1801 #rank로 사용하기 위해 +1801
sell_stock.columns = ['순위','종목코드']
sell_stock

,순위,종목코드
0,1801,048260
1,1802,003920
2,1803,051900
3,1804,007310
4,1805,298020
...,...,...
195,1996,173130
196,1997,007700
197,1998,139050
198,1999,108490


In [ ]:
buy_sell_stock = pd.concat([buy_stock, sell_stock], axis = 0)
buy_sell_stock

,순위,종목코드
0,1,102120
1,2,094970
2,3,002240
3,4,008040
4,5,037330
...,...,...
195,1996,173130
196,1997,007700
197,1998,139050
198,1999,108490


In [ ]:
#그 외의 종목들 더미로 생성
dummy_stock_code = []
for code in unique_codes:
    if code not in list(buy_sell_stock['종목코드']):
        dummy_stock_code.append(code)
len(dummy_stock_code)

1600

In [ ]:
dummy_stock = pd.DataFrame({'순위':[i for i in range(201,1801)],'종목코드':dummy_stock_code})
dummy_stock

,순위,종목코드
0,201,005930
1,202,000660
2,203,000270
3,204,035420
4,205,005490
...,...,...
1595,1796,031510
1596,1797,030350
1597,1798,081580
1598,1799,115610


In [ ]:
VWAP_result_df = pd.concat([buy_sell_stock, dummy_stock], axis = 0,ignore_index=True)
VWAP_result_df['종목코드'] = 'A'+VWAP_result_df['종목코드']
VWAP_result_df

,순위,종목코드
0,1,A102120
1,2,A094970
2,3,A002240
3,4,A008040
4,5,A037330
...,...,...
1995,1796,A031510
1996,1797,A030350
1997,1798,A081580
1998,1799,A115610


In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [ ]:
vwap_submission = sample_submission[['종목코드']].merge(VWAP_result_df[['종목코드', '순위']], on='종목코드', how='left')
vwap_submission[vwap_submission['종목코드'] == 'A003280']

,종목코드,순위
146,A003280,112


In [ ]:
vwap_submission.to_csv('vwap_submission0730.csv', index=False)